In [139]:
import pandas as pd

### Reading in COVID-19 datasets

In [154]:
confirmed = pd.read_csv('./data/covid_confirmed_usafacts.csv')
deaths = pd.read_csv('./data/covid_deaths_usafacts.csv')
county = pd.read_csv('./data/covid_county_population_usafacts.csv')

confirmed.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,19913,19913,19913,19913,19913,19913,19913,19913,19913,19913
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,70521,70521,70521,70521,70521,70521,70521,70521,70521,70521
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,7582,7582,7582,7582,7582,7582,7582,7582,7582,7582
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,8149,8149,8149,8149,8149,8149,8149,8149,8149,8149


In [141]:
deaths.head()

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,235,235,235,235,235,235,235,235,235,235
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,731,731,731,731,731,731,731,731,731,731
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,111,111,111,111,111,111,111,111,111,111


In [155]:
county = pd.read_csv('./data/covid_county_population_usafacts.csv')
county.head()

,countyFIPS,County Name,State,population
0,0,Statewide Unallocated,AL,0
1,1001,Autauga County,AL,55869
2,1003,Baldwin County,AL,223234
3,1005,Barbour County,AL,24686
4,1007,Bibb County,AL,22394


### Filtering to only last seven days of the data

In [143]:
confirmed_last7 = pd.concat([confirmed.iloc[:, :4], confirmed.iloc[:, -7:]], axis=1)
deaths_last7 = pd.concat([deaths.iloc[:, :4], deaths.iloc[:, -7:]], axis=1)

### Matching dtypes for merging

In [144]:
for df in [confirmed_last7, deaths_last7, county]:
    df['countyFIPS'] = df['countyFIPS'].astype(str).str.zfill(5)
    df['County Name'] = df['County Name'].str.strip()
    df['State'] = df['State'].str.strip().str.upper()

### Merging datasets on a left join & dropping duplicate columm

In [145]:
confirmed_county = pd.merge(confirmed_last7, 
                            county[['countyFIPS', 'County Name', 'State', 'population']],
                            on=['countyFIPS', 'County Name', 'State'],
                            how='left')

covid_superset = pd.merge(confirmed_county, deaths_last7, on=['countyFIPS', 'County Name', 'State'], how='left')
covid_superset.drop(['StateFIPS_y'], axis=1, inplace=True)
covid_superset.rename(columns={'StateFIPS_x':'StateFIPS'}, inplace=True)
covid_superset.sort_values(by='StateFIPS', inplace=True)
covid_superset

,countyFIPS,County Name,State,StateFIPS,2023-07-17_x,2023-07-18_x,2023-07-19_x,2023-07-20_x,2023-07-21_x,2023-07-22_x,2023-07-23_x,population,2023-07-17_y,2023-07-18_y,2023-07-19_y,2023-07-20_y,2023-07-21_y,2023-07-22_y,2023-07-23_y
0,00000,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
36,01071,Jackson County,AL,1,18504,18504,18504,18504,18504,18504,18504,51626.0,257,257,257,257,257,257,257
37,01073,Jefferson County,AL,1,240736,240736,240736,240736,240736,240736,240736,658573.0,2536,2536,2536,2536,2536,2536,2536
38,01075,Lamar County,AL,1,4819,4819,4819,4819,4819,4819,4819,13805.0,74,74,74,74,74,74,74
39,01077,Lauderdale County,AL,1,30743,30743,30743,30743,30743,30743,30743,92729.0,423,423,423,423,423,423,423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170,56001,Albany County,WY,56,11661,11661,11661,11661,11661,11661,11661,38880.0,54,54,54,54,54,54,54
3169,00000,Statewide Unallocated,WY,56,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
3191,56043,Washakie County,WY,56,2640,2640,2640,2640,2640,2640,2640,7805.0,51,51,51,51,51,51,51
3179,56019,Johnson County,WY,56,2404,2404,2404,2404,2404,2404,2404,8445.0,22,22,22,22,22,22,22


### Filling NaN values with 0 and 'Unknown'

In [146]:
fill_zero= covid_superset.select_dtypes(include=['float']).columns
covid_superset[fill_zero] = covid_superset[fill_zero].fillna(0)

covid_superset['StateFIPS'] = covid_superset['StateFIPS'].fillna('Unknown')

### Adding _cases & _deaths to dates

In [147]:
covid_superset.columns = covid_superset.columns.str.replace('_x', '_cases', regex=False)
covid_superset.columns = covid_superset.columns.str.replace('_y', '_deaths', regex=False)
covid_superset

,countyFIPS,County Name,State,StateFIPS,2023-07-17_cases,2023-07-18_cases,2023-07-19_cases,2023-07-20_cases,2023-07-21_cases,2023-07-22_cases,2023-07-23_cases,population,2023-07-17_deaths,2023-07-18_deaths,2023-07-19_deaths,2023-07-20_deaths,2023-07-21_deaths,2023-07-22_deaths,2023-07-23_deaths
0,00000,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
36,01071,Jackson County,AL,1,18504,18504,18504,18504,18504,18504,18504,51626.0,257,257,257,257,257,257,257
37,01073,Jefferson County,AL,1,240736,240736,240736,240736,240736,240736,240736,658573.0,2536,2536,2536,2536,2536,2536,2536
38,01075,Lamar County,AL,1,4819,4819,4819,4819,4819,4819,4819,13805.0,74,74,74,74,74,74,74
39,01077,Lauderdale County,AL,1,30743,30743,30743,30743,30743,30743,30743,92729.0,423,423,423,423,423,423,423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170,56001,Albany County,WY,56,11661,11661,11661,11661,11661,11661,11661,38880.0,54,54,54,54,54,54,54
3169,00000,Statewide Unallocated,WY,56,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
3191,56043,Washakie County,WY,56,2640,2640,2640,2640,2640,2640,2640,7805.0,51,51,51,51,51,51,51
3179,56019,Johnson County,WY,56,2404,2404,2404,2404,2404,2404,2404,8445.0,22,22,22,22,22,22,22


### Moving population to the end of the dataframe

In [152]:
col = covid_superset.pop('population')
covid_superset.insert(18, col.name, col)

covid_superset.sort_values(by='StateFIPS', inplace=True)

covid_superset['population'] = covid_superset['population'].astype('int64')
covid_superset

,countyFIPS,County Name,State,StateFIPS,2023-07-17_cases,2023-07-18_cases,2023-07-19_cases,2023-07-20_cases,2023-07-21_cases,2023-07-22_cases,2023-07-23_cases,2023-07-17_deaths,2023-07-18_deaths,2023-07-19_deaths,2023-07-20_deaths,2023-07-21_deaths,2023-07-22_deaths,2023-07-23_deaths,population
0,00000,Statewide Unallocated,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,01013,Butler County,AL,1,6617,6617,6617,6617,6617,6617,6617,132,132,132,132,132,132,132,19448
1,01001,Autauga County,AL,1,19913,19913,19913,19913,19913,19913,19913,235,235,235,235,235,235,235,55869
2,01003,Baldwin County,AL,1,70521,70521,70521,70521,70521,70521,70521,731,731,731,731,731,731,731,223234
3,01005,Barbour County,AL,1,7582,7582,7582,7582,7582,7582,7582,104,104,104,104,104,104,104,24686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3190,56041,Uinta County,WY,56,6468,6468,6468,6468,6468,6468,6468,43,43,43,43,43,43,43,20226
3181,56023,Lincoln County,WY,56,4927,4927,4927,4927,4927,4927,4927,38,38,38,38,38,38,38,19830
3172,56005,Campbell County,WY,56,14064,14064,14064,14064,14064,14064,14064,167,167,167,167,167,167,167,46341
3185,56031,Platte County,WY,56,2313,2313,2313,2313,2313,2313,2313,46,46,46,46,46,46,46,8393


In [153]:
covid_superset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3193 entries, 0 to 3192
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   countyFIPS         3193 non-null   object
 1   County Name        3193 non-null   object
 2   State              3193 non-null   object
 3   StateFIPS          3193 non-null   int64 
 4   2023-07-17_cases   3193 non-null   int64 
 5   2023-07-18_cases   3193 non-null   int64 
 6   2023-07-19_cases   3193 non-null   int64 
 7   2023-07-20_cases   3193 non-null   int64 
 8   2023-07-21_cases   3193 non-null   int64 
 9   2023-07-22_cases   3193 non-null   int64 
 10  2023-07-23_cases   3193 non-null   int64 
 11  2023-07-17_deaths  3193 non-null   int64 
 12  2023-07-18_deaths  3193 non-null   int64 
 13  2023-07-19_deaths  3193 non-null   int64 
 14  2023-07-20_deaths  3193 non-null   int64 
 15  2023-07-21_deaths  3193 non-null   int64 
 16  2023-07-22_deaths  3193 non-null   int64 
 17  

### Exporting dataframe as CSV

In [157]:
# covid_superset.to_csv('./data/covid_superset.csv', index=False)  